In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import keras.backend as K
# %pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [29]:
df = pd.read_csv('../../Dataset/cleaned_used_cars_v1.csv')

In [30]:
df.shape

(207735, 12)

In [31]:
# dummies = pd.get_dummies(df.drop(['dateCrawled','lastSeen','dateCreated','name'], axis=1), columns=['seller', 'offerType','abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage'])
# df_preprocessed = pd.concat([df.drop(['price','index','dateCrawled','lastSeen','yearOfRegistration','monthOfRegistration','nrOfPictures','postalCode',
#              'name','seller','offerType','abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage','dateCreated'], axis=1),
#                              dummies], axis=1)
onehot_columns = ['abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage']
onehot_df = df[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
score_onehot_drop = df.drop(onehot_columns, axis = 1)
score_onehot = pd.concat([score_onehot_drop, onehot_df], axis = 1)
df_preprocessed = score_onehot

In [32]:
df_preprocessed.shape

(207735, 298)

In [33]:
df_preprocessed

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,abtest_control,abtest_test,vehicleType_compact car,vehicleType_convertible,vehicleType_other,...,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_no,notRepairedDamage_yes
0,1500,2001,75,150000,6,0,1,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,3600,2008,69,90000,7,0,1,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,650,1995,102,150000,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2200,2004,109,150000,8,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1980,50,40000,7,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207730,7900,2010,140,150000,7,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
207731,3999,2005,3,150000,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
207732,3200,2004,225,150000,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
207733,1199,2000,101,125000,3,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [34]:
def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [35]:
X = df_preprocessed.astype(float).to_numpy()
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(1))

print('--------------------Compile------------------------------------')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae', r2])

# early_stop = EarlyStopping(monitor='val_loss', patience=100)

print('--------------------Fit------------------------------------')
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=1000, epochs=100)

print('--------------------Evaluate------------------------------------')
score = model.evaluate(X_test, y_test)
print(f'Deep Learning Regression Mean Absolute Error: {score[1]:.2f}')
print(f'Deep Learning Regression R-squared Score: {score[2]:.2f}')

--------------------Compile------------------------------------
--------------------Fit------------------------------------
Epoch 1/100
133/133 [==============================] - 3s 9ms/step - loss: 144554967040.0000 - mae: 6501.2490 - r2: -0.1719 - val_loss: 2931755264.0000 - val_mae: 5112.8677 - val_r2: -0.1058
Epoch 2/100
133/133 [==============================] - 1s 7ms/step - loss: 140285853696.0000 - mae: 6150.2710 - r2: -0.0584 - val_loss: 2811236608.0000 - val_mae: 4784.3115 - val_r2: 0.0021
Epoch 3/100
133/133 [==============================] - 1s 6ms/step - loss: 134637150208.0000 - mae: 5796.4707 - r2: 0.0539 - val_loss: 2695835136.0000 - val_mae: 4411.0190 - val_r2: 0.1152
Epoch 4/100
133/133 [==============================] - 1s 6ms/step - loss: 129221705728.0000 - mae: 5300.6304 - r2: 0.1777 - val_loss: 2567286784.0000 - val_mae: 3751.3311 - val_r2: 0.2615
Epoch 5/100
133/133 [==============================] - 1s 6ms/step - loss: 122941169664.0000 - mae: 4508.3491 - r2: 0

In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                19136     
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 21,761
Trainable params: 21,761
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.get_weights()

[array([[ 0.07756545,  0.18043354, -0.08142801, ...,  0.14166184,
          0.11268731,  0.04504988],
        [-0.07078474,  0.18117018,  0.0650039 , ...,  0.04124867,
          0.13440739,  0.07045129],
        [-0.02478414,  0.00360626, -0.06046724, ...,  0.1731681 ,
          0.03924756,  0.05413713],
        ...,
        [-0.22880995, -0.13888729,  0.02919753, ...,  0.10449643,
          0.02917315,  0.26132375],
        [ 0.08737887, -0.00719999, -0.05727562, ...,  0.16037567,
          0.07101771,  0.17898001],
        [-0.00047263, -0.16250099,  0.02253671, ..., -0.05105543,
          0.05547773,  0.10153022]], dtype=float32),
 array([ 0.01330154,  0.07930525,  0.        ,  0.        ,  0.06146939,
         0.04846936,  0.07628948,  0.01454827, -0.02135238,  0.        ,
        -0.06780432,  0.00459745, -0.01659564,  0.15276518,  0.00578983,
        -0.0185874 ,  0.07646849, -0.00760064,  0.05748874,  0.09296624,
         0.03563549,  0.15496969, -0.07111946,  0.05539704,  0.040

In [38]:
predictions = model.predict(X_test)

# Print the predicted prices
print(predictions)

1299/1299 [==============================] - 2s 2ms/step
[[141644.22  ]
 [  1080.4426]
 [  4453.525 ]
 ...
 [  2175.138 ]
 [  6447.228 ]
 [ 17506.455 ]]


In [39]:
y_test

120328    145000
34901       1095
165437      4480
155406      1325
24030       1800
           ...  
155498      6700
93180        650
153338      2200
131690      6500
127733     17799
Name: price, Length: 41547, dtype: int64

In [40]:
import plotly.express as px
import pandas as pd

# create a dataframe with the training history
df_graph = pd.DataFrame(history.history)

# create line charts for loss and MAE
fig = px.line(df_graph, x=df_graph.index+1, y=['loss', 'val_loss'], labels={'x':'Epoch', 'value':'Loss'},
              title='Training and Validation Loss')
fig.update_traces(mode='lines')
fig.show()

fig = px.line(df_graph, x=df_graph.index+1, y=['mae', 'val_mae'], labels={'x':'Epoch', 'value':'MAE'},
              title='Training and Validation MAE')
fig.update_traces(mode='lines')
fig.show()